# PRISM Temporal Analysis

Analyze how indicator importance changes over rolling time windows.

**Key Features:**
- Rolling 1-year window analysis
- Track rank changes over time
- Multiple visualization types
- Optimized for 50-60+ indicators

**Performance Profiles:**
- `chromebook`: Fast, uses fewer lenses (3), larger steps
- `standard`: Balanced (4 lenses, monthly steps)
- `powerful`: High resolution (5 lenses, bi-weekly steps)

## Step 1: Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set path to your prism-engine folder
PRISM_PATH = '/content/drive/MyDrive/prism-engine/prism-engine'

import sys
sys.path.insert(0, PRISM_PATH)
sys.path.insert(0, f'{PRISM_PATH}/05_engine/orchestration')
sys.path.insert(0, f'{PRISM_PATH}/08_visualization/plotters')

print(f"PRISM path: {PRISM_PATH}")

In [ ]:
# Load the temporal runner
exec(open(f'{PRISM_PATH}/Start/temporal_runner.py').read())
print("Temporal runner loaded!")

## Step 2: Load and Clean Data

In [ ]:
import pandas as pd
import numpy as np

# Load master panel
panel = pd.read_csv(f'{PRISM_PATH}/data/raw/master_panel.csv', 
                    index_col=0, parse_dates=True)

print(f"Loaded: {panel.shape}")
print(f"Date range: {panel.index[0]} to {panel.index[-1]}")
print(f"Columns: {list(panel.columns)}")

In [ ]:
# Clean the data
panel_clean = panel.ffill().bfill().dropna()

print(f"After cleaning: {panel_clean.shape}")
print(f"NaNs: {panel_clean.isna().sum().sum()}")

## Step 3: Run Temporal Analysis

Choose your performance profile:
- `'chromebook'` - Fast, lower resolution
- `'standard'` - Balanced
- `'powerful'` - High resolution (Mac Mini)

In [ ]:
# Run temporal analysis
# Change 'standard' to 'chromebook' for faster execution
results = run_temporal_analysis(panel_clean, profile='standard')

In [ ]:
# Get summary
summary = get_summary(results)
print("\nTOP 15 BY CURRENT RANK:")
print(summary.head(15))

## Step 4: Visualize Ranking Evolution

In [ ]:
# Line chart: How top indicator ranks change over time
from temporal_plots import plot_ranking_evolution
plot_ranking_evolution(results, top_n=10)

In [ ]:
# Bump chart: Rank trajectories
from temporal_plots import plot_bump_chart
plot_bump_chart(results, top_n=12)

In [ ]:
# Heatmap: All ranks over time
from temporal_plots import plot_rank_heatmap
plot_rank_heatmap(results, top_n=20)

In [ ]:
# Recent rank changes
from temporal_plots import plot_rank_changes
plot_rank_changes(results, lookback=6)

In [ ]:
# Stability analysis
from temporal_plots import plot_stability_analysis
plot_stability_analysis(results, top_n=20)

## Step 5: Find Trending Indicators

In [ ]:
# Find rising and falling indicators
trending = find_trending(results, lookback=6)

print(f"Period: {trending['period']}")
print()
print("RISING (improving in importance):")
for ind, change in list(trending['rising'].items())[:10]:
    print(f"  {ind}: +{change:.1f} ranks")

print()
print("FALLING (declining in importance):")
for ind, change in list(trending['falling'].items())[:10]:
    print(f"  {ind}: {change:.1f} ranks")

## Step 6: Save All Visualizations

In [ ]:
# Save all plots to a folder
output_dir = f'{PRISM_PATH}/06_output/temporal_analysis'
saved = generate_all_plots(results, output_dir=output_dir)

print("\nSaved files:")
for plot_type, path in saved.items():
    print(f"  {plot_type}: {path}")

## Step 7: Export Data

In [ ]:
# Export rankings over time to CSV
from temporal_analysis import TemporalPRISM

temporal = TemporalPRISM(panel_clean)
rank_df = temporal.get_ranking_dataframe(results)

rank_df.to_csv(f'{output_dir}/rankings_over_time.csv')
print(f"Saved rankings to {output_dir}/rankings_over_time.csv")

# Show sample
rank_df.tail()

## Optional: Create Animation Frames

In [ ]:
# Create frames for animation (optional - takes time)
# Uncomment to run:

# from temporal_plots import create_animation_frames
# frames_dir = f'{output_dir}/animation_frames'
# frame_paths = create_animation_frames(results, frames_dir, top_n=15)

---

## Quick Reference

### Performance Tips for 50-60+ Indicators:

1. **Chromebook (Limited RAM)**:
   ```python
   results = run_temporal_analysis(panel_clean, profile='chromebook')
   ```

2. **Mac Mini (Powerful)**:
   ```python
   results = run_temporal_analysis(panel_clean, profile='powerful')
   ```

3. **Very Long Time Series**:
   ```python
   from temporal_analysis import StreamingTemporalPRISM
   streaming = StreamingTemporalPRISM(panel_clean, 'output.csv')
   streaming.run_streaming()
   ```

### Customization:

```python
results = run_temporal_analysis(
    panel_clean,
    window_years=2.0,      # 2-year rolling window
    step_months=0.5,       # Bi-weekly steps
    lenses=['magnitude', 'pca', 'influence', 'clustering', 'decomposition']
)
```